# Using [vtreat](https://github.com/WinVector/pyvtreat) with Multinomial Classification Problems

Nina Zumel and John Mount
September 2019

Note: this is a description of the [`Python` version of `vtreat`](https://github.com/WinVector/pyvtreat), the same example for the [`R` version of `vtreat`](https://github.com/WinVector/vtreat) can be found [here](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/Multinomial.md).


## Preliminaries

Load modules/packages.

In [1]:
import pkg_resources
import pandas
import numpy
import numpy.random
import seaborn
import matplotlib.pyplot as plt
import vtreat
import vtreat.util
import wvpy.util

Generate example data. 

* `y` is a noisy sinusoidal function of the variable `x`
* `yc` is the multiple class output to be predicted: : `y`'s quantized value. 
* Input `xc` is a categorical variable that represents a discretization of `y`, along some `NaN`s
* Input `x2` is a pure noise variable with no relationship to the output

In [2]:
def make_data(nrows):
    d = pandas.DataFrame({'x': 5*numpy.random.normal(size=nrows)})
    d['y'] = numpy.sin(d['x']) + 0.1*numpy.random.normal(size=nrows)
    d.loc[numpy.arange(3, 10), 'x'] = numpy.nan                           # introduce a nan level
    d['xc'] = ['level_' + str(5*numpy.round(yi/5, 1)) for yi in d['y']]
    d['x2'] = numpy.random.normal(size=nrows)
    d.loc[d['xc']=='level_-1.0', 'xc'] = numpy.nan  # introduce a nan level
    d['yc'] = numpy.where(d['y']>0.5, 'large', numpy.where(d['y']<-0.5, 'small', 'liminal'))
    return d

d = make_data(500)

d.head()

,x,y,xc,x2,yc
0,2.700607,0.268432,level_0.5,-1.696959,liminal
1,-3.409446,0.085874,level_0.0,-0.064905,liminal
2,-2.497962,-0.714933,level_-0.5,-0.743320,small
3,NaN,-0.060780,level_-0.0,-2.008286,liminal
4,NaN,1.085506,level_1.0,0.431998,large


### Some quick data exploration

Check how many levels `xc` has, and their distribution (including `NaN`)

In [3]:
d['xc'].unique()

array(['level_0.5', 'level_0.0', 'level_-0.5', 'level_-0.0', 'level_1.0',
       nan], dtype=object)

In [4]:
d['xc'].value_counts(dropna=False)

level_-0.5    113
level_1.0     108
NaN            98
level_0.5      88
level_0.0      52
level_-0.0     41
Name: xc, dtype: int64

Show the distribution of `yc`

In [5]:
d['yc'].value_counts(dropna=False)

liminal    180
small      169
large      151
Name: yc, dtype: int64

Plot conditional distribution of `x` versus `yc`.

In [6]:
# TODO: multiple histograms ( generalize wvpy dual_density_plot )

## Build a transform appropriate for classification problems.

Now that we have the data, we want to treat it prior to modeling: we want training data where all the input variables are numeric and have no missing values or `NaN`s.

First create the data treatment transform object, in this case a treatment for a binomial classification problem.

In [7]:
transform = vtreat.MultinomialOutcomeTreatment(
    outcome_name='yc',    # outcome variable
    cols_to_copy=['y'],   # columns to "carry along" but not treat as input variables
)  

Use the training data `d` to fit the transform and the return a treated training set: completely numeric, with no missing values.
Note that for the training data `d`: `transform.fit_transform()` is **not** the same as `transform.fit().transform()`; the second call can lead to nested model bias in some situations, and is **not** recommended. 
For other, later data, not seen during transform design `transform.transform(o)` is an appropriate step.

In [8]:
d_prepared = transform.fit_transform(d, d['yc'])

Now examine the score frame, which gives information about each new variable, including its type, which original variable it is  derived from, its (cross-validated) correlation with the outcome, and its (cross-validated) significance as a one-variable linear model for the outcome. 

In [9]:
transform.score_frame_

,variable,orig_variable,treatment,y_aware,has_range,PearsonR,significance,vcount,default_threshold,recommended,outcome_target
0,x_is_bad,x,missing_indicator,False,True,-0.004227,9.248901e-01,2.0,0.100000,False,large
1,xc_is_bad,xc,missing_indicator,False,True,-0.324770,9.570226e-14,2.0,0.100000,True,large
2,x,x,clean_copy,False,True,0.060607,1.760313e-01,2.0,0.100000,False,large
3,x2,x2,clean_copy,False,True,0.015279,7.332465e-01,2.0,0.100000,False,large
4,xc_logit_code_large,xc,logit_code,True,True,0.841585,2.567633e-135,3.0,0.066667,True,large
5,xc_logit_code_liminal,xc,logit_code,True,True,-0.409457,1.236682e-21,3.0,0.066667,False,large
6,xc_logit_code_small,xc,logit_code,True,True,-0.563718,2.803242e-43,3.0,0.066667,False,large
7,xc_prevalence_code,xc,prevalence_code,False,True,0.281141,1.550617e-10,1.0,0.200000,True,large
8,xc_lev_level_-0.5,xc,indicator_code,False,True,-0.355434,2.463731e-16,5.0,0.040000,True,large
9,xc_lev_level_1.0,xc,indicator_code,False,True,0.797982,1.362405e-111,5.0,0.040000,True,large


Note that the variable `xc` has been converted to multiple variables: 

* an indicator variable for each possible level (`xc_lev_level_*`)
* the value of a (cross-validated) one-variable model for `yc` as a function of `xc` (`xc_logit_code`)
* a variable that returns how prevalent this particular value of `xc` is in the training data (`xc_prevalence_code`)
* a variable indicating when `xc` was `NaN` in the original data (`xc_is_bad`, `x_is_bad`)

Any or all of these new variables are available for downstream modeling.

The `recommended` column indicates which variables are non constant (`has_range` == True) and have a significance value smaller than `default_threshold`. See the section *Deriving the Default Thresholds* below for the reasoning behind the default thresholds. Recommended columns are intended as advice about which variables appear to be most likely to be useful in a downstream model. This advice attempts to be conservative, to reduce the possibility of mistakenly eliminating variables that may in fact be useful (although, obviously, it can still mistakenly eliminate variables that have a real but non-linear relationship to the output, as is the case with `x`, in  our example).

## Examining variables

To select variables we either make our selection in terms of new variables as follows.

In [10]:
score_frame = transform.score_frame_
good_new_variables = score_frame.variable[score_frame.recommended].unique()
good_new_variables

array(['xc_is_bad', 'xc_logit_code_large', 'xc_prevalence_code',
       'xc_lev_level_-0.5', 'xc_lev_level_1.0', 'xc_lev__NA_',
       'xc_lev_level_0.5', 'xc_lev_level_0.0', 'xc_logit_code_liminal',
       'xc_logit_code_small'], dtype=object)

Or in terms of original variables as follows.

In [11]:
good_original_variables = score_frame.orig_variable[score_frame.recommended].unique()
good_original_variables

array(['xc'], dtype=object)

Notice, in each case we must call unique as each variable (derived or original) is potentially qualified against each possible outcome.

Notice that, by default, `d_prepared` only includes recommended variables (along with `y` and `yc`):

In [12]:
d_prepared.head()

,y,yc,xc_is_bad,xc_logit_code_large,xc_logit_code_liminal,xc_logit_code_small,xc_prevalence_code,xc_lev_level_-0.5,xc_lev_level_1.0,xc_lev__NA_,xc_lev_level_0.5,xc_lev_level_0.0
0,0.268432,liminal,0.0,0.423869,0.350345,-5.762421,0.176,0.0,0.0,0.0,1.0,0.0
1,0.085874,liminal,0.0,-5.641837,0.985762,-5.775338,0.104,0.0,0.0,0.0,0.0,1.0
2,-0.714933,small,0.0,-5.628960,-0.002235,0.585699,0.226,1.0,0.0,0.0,0.0,0.0
3,-0.060780,liminal,0.0,-5.710410,1.033714,-5.728814,0.082,0.0,0.0,0.0,0.0,0.0
4,1.085506,large,0.0,1.137272,-5.856069,-5.768219,0.216,0.0,1.0,0.0,0.0,0.0


This is `vtreat`s default behavior; to include all variables in the prepared data, set the parameter `filter_to_recommended` to False, as we show later, in the *Parameters for `BinomialOutcomeTreatment`* section below.


## A Closer Look at `logit_code` variables

Variables of type `logit_code` are the outputs of a one-variable hierarchical logistic regression of a categorical variable (in our example, `xc`) against the centered output on the (cross-validated) treated training data. 

Let's see whether `xc_logit_code` makes a good one-variable model for `yc`. It has a large AUC:

In [13]:
# TODO: figure out a presentation
# wvpy.util.plot_roc(prediction=d_prepared['xc_logit_code'], 
#                   istrue=d_prepared['yc'],
#                   title = 'performance of xc_logit_code variable')

This indicates that `xc_logit_code` is strongly predictive of the outcome. Negative values of `xc_logit_code` correspond strongly to negative outcomes, and positive values correspond strongly to positive outcomes.

In [14]:
# TODO: figure out a presentation
# wvpy.util.dual_density_plot(probs=d_prepared['xc_logit_code'], 
#                            istrue=d_prepared['yc'])

The values of `xc_logit_code_*` are in "link space". We can often visualize the relationship a little better by converting the logistic score to a probability.

In [15]:
from scipy.special import expit  # sigmoid
from scipy.special import logit

# TODO: figure out presenetation
# offset = logit(numpy.mean(d_prepared.yc))
# wvpy.util.dual_density_plot(probs=expit(d_prepared['xc_logit_code_large'] + offset),
#                            istrue=d_prepared['yc'])                                   

Variables of type `logit_code` are useful when dealing with categorical variables with a very large number of possible levels. For example, a categorical variable with 10,000 possible values potentially converts to 10,000 indicator variables, which may be unwieldy for some modeling methods. Using a single numerical variable of type `logit_code` may be a preferable alternative.

## Using the Prepared Data in a Model

Of course, what we really want to do with the prepared training data is to fit a model jointly with all the (recommended) variables. 
Let's try fitting a logistic regression model to `d_prepared`.

In [16]:
import sklearn.linear_model
import seaborn

not_variables = ['y', 'yc', 'prediction', 'prob_on_predicted_class', 'predict', 'large', 'liminal', 'small']
model_vars = [v for v in d_prepared.columns if v not in set(not_variables)]

fitter = sklearn.linear_model.LogisticRegression(
    solver = 'saga',
    penalty = 'l2',
    C = 1,
    max_iter = 1000,
    multi_class = 'multinomial')
fitter.fit(d_prepared[model_vars], d_prepared['yc'])

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
def add_predictions(d_prepared, model_vars, fitter):
    pred = fitter.predict_proba(d_prepared[model_vars])
    classes = fitter.classes_
    d_prepared['prob_on_predicted_class'] = 0
    d_prepared['predict'] = None
    for i in range(len(classes)):
        cl = classes[i]
        d_prepared[cl] = pred[:, i]
        improved = d_prepared[cl] > d_prepared['prob_on_predicted_class']
        d_prepared.loc[improved, 'predict'] = cl
        d_prepared.loc[improved, 'prob_on_predicted_class'] = d_prepared.loc[improved, cl]
    return d_prepared

def add_value_by_column(d_prepared, name_column, new_column):
    vals = d[name_column].unique()
    d_prepared[new_column] = None
    for v in vals:
        matches = d_prepared[name_column]==v
        d_prepared.loc[matches, new_column] = d_prepared.loc[matches, v]
    return d_prepared

In [18]:
# now predict
d_prepared = add_predictions(d_prepared, model_vars, fitter)
d_prepared = add_value_by_column(d_prepared, 'yc', 'prob_on_correct_class')
d_prepared.head()

,y,yc,xc_is_bad,xc_logit_code_large,xc_logit_code_liminal,xc_logit_code_small,xc_prevalence_code,xc_lev_level_-0.5,xc_lev_level_1.0,xc_lev__NA_,xc_lev_level_0.5,xc_lev_level_0.0,prob_on_predicted_class,predict,large,liminal,small,prob_on_correct_class
0,0.268432,liminal,0.0,0.423869,0.350345,-5.762421,0.176,0.0,0.0,0.0,1.0,0.0,0.529168,liminal,0.470418,0.529168,0.000413,0.529168
1,0.085874,liminal,0.0,-5.641837,0.985762,-5.775338,0.104,0.0,0.0,0.0,0.0,1.0,0.998639,liminal,0.000575,0.998639,0.000786,0.998639
2,-0.714933,small,0.0,-5.628960,-0.002235,0.585699,0.226,1.0,0.0,0.0,0.0,0.0,0.627012,small,0.000332,0.372656,0.627012,0.627012
3,-0.060780,liminal,0.0,-5.710410,1.033714,-5.728814,0.082,0.0,0.0,0.0,0.0,0.0,0.998560,liminal,0.000557,0.998560,0.000884,0.99856
4,1.085506,large,0.0,1.137272,-5.856069,-5.768219,0.216,0.0,1.0,0.0,0.0,0.0,0.998962,large,0.998962,0.000828,0.000209,0.998962


In [19]:
# TODO: work out a presentation
# Confusion matrix: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# also maybe density plot of prob_on_correct_class
# look at the ROC curve (on the training data)
#wvpy.util.plot_roc(prediction=d_prepared['prediction'], 
#                   istrue=d_prepared['yc'],
#                   title = 'Performance of logistic regression model on training data')

Now apply the model to new data.

In [20]:
# create the new data
dtest = make_data(450)

# prepare the new data with vtreat
dtest_prepared = transform.transform(dtest)

# apply the model to the prepared data
dtest_prepared = add_predictions(dtest_prepared, model_vars, fitter)
dtest_prepared = add_value_by_column(dtest_prepared, 'yc', 'prob_on_correct_class')
dtest_prepared.head()

,y,yc,xc_is_bad,xc_logit_code_large,xc_logit_code_liminal,xc_logit_code_small,xc_prevalence_code,xc_lev_level_-0.5,xc_lev_level_1.0,xc_lev__NA_,xc_lev_level_0.5,xc_lev_level_0.0,prob_on_predicted_class,predict,large,liminal,small,prob_on_correct_class
0,-0.484566,liminal,0.0,-5.698433,0.031524,0.613593,0.226,1.0,0.0,0.0,0.0,0.0,0.628341,small,0.000295,0.371364,0.628341,0.371364
1,-0.813218,small,1.0,-5.696088,-5.870838,1.082719,0.196,0.0,0.0,1.0,0.0,0.0,0.998723,small,0.000326,0.000951,0.998723,0.998723
2,-0.900069,small,1.0,-5.696088,-5.870838,1.082719,0.196,0.0,0.0,1.0,0.0,0.0,0.998723,small,0.000326,0.000951,0.998723,0.998723
3,1.059184,large,0.0,1.194985,-5.872510,-5.809646,0.216,0.0,1.0,0.0,0.0,0.0,0.999052,large,0.999052,0.000762,0.000186,0.999052
4,-0.431252,liminal,0.0,-5.698433,0.031524,0.613593,0.226,1.0,0.0,0.0,0.0,0.0,0.628341,small,0.000295,0.371364,0.628341,0.371364


In [21]:
wvpy.util.plot_roc(prediction=dtest_prepared['prediction'], 
                   istrue=dtest_prepared['yc'],
                   title = 'Performance of logistic regression model on test data')

KeyError: 'prediction'

## Parameters for `BinomialOutcomeTreatment`

We've tried to set the defaults for all parameters so that `vtreat` is usable out of the box for most applications.


In [ ]:
vtreat.vtreat_parameters()

**use_hierarchical_estimate:**: When True, uses hierarchical smoothing when estimating `logit_code` variables; when False, uses unsmoothed logistic regression.

**coders**: The types of synthetic variables that `vtreat` will (potentially) produce. See *Types of prepared variables* below.

**filter_to_recommended**: When True, prepared data only includes variables marked as "recommended" in score frame. When False, prepared data includes all variables. See the Example below.

**indicator_min_fraction**: For categorical variables, indicator variables (type `indicator_code`) are only produced for levels that are present at least `indicator_min_fraction` of the time. A consequence of this is that 1/`indicator_min_fraction` is the maximum number of indicators that will be produced for a given categorical variable. To make sure that *all* possible indicator variables are produced, set `indicator_min_fraction = 0`

**cross_validation_plan**: The cross validation method used by `vtreat`. Most people won't have to change this.

**cross_validation_k**: The number of folds to use for cross-validation

**user_transforms**: For passing in user-defined transforms for custom data preparation. Won't be needed in most situations, but see [here](https://github.com/WinVector/pyvtreat/blob/master/Examples/UserCoders/UserCoders.ipynb) for an example of applying a GAM transform to input variables.

**sparse_indicators**: When True, use a (Pandas) sparse representation for indicator variables. This representation is compatible with `sklearn`; however, it may not be compatible with other modeling packages. When False, use a dense representation.

### Example: Use all variables to model, not just recommended

In [ ]:
transform_all = vtreat.BinomialOutcomeTreatment(
    outcome_name='yc',    # outcome variable
    outcome_target=True,  # outcome of interest
    cols_to_copy=['y'],   # columns to "carry along" but not treat as input variables
    params = vtreat.vtreat_parameters({
        'filter_to_recommended': False
    })
)  

transform_all.fit_transform(d, d['yc']).columns

In [ ]:
transform_all.score_frame_

Note that the prepared data produced by `fit_transform()` includes all the variables, including those that were not marked as "recommended". 

## Types of prepared variables

**clean_copy**: Produced from numerical variables: a clean numerical variable with no `NaNs` or missing values

**indicator_code**: Produced from categorical variables, one for each (common) level: for each level of the variable, indicates if that level was "on"

**prevalence_code**: Produced from categorical variables: indicates how often each level of the variable was "on"

**logit_code**: Produced from categorical variables: score from a one-dimensional model of the centered output as a function of the variable

**missing_indicator**: Produced for both numerical and categorical variables: an indicator variable that marks when the original variable was missing or  `NaN`

**deviation_code**: not used by `BinomialOutcomeTreatment`

**impact_code**: not used by `BinomialOutcomeTreatment`

### Example: Produce only a subset of variable types

In this example, suppose you only want to use indicators and continuous variables in your model; 
in other words, you only want to use variables of types (`clean_copy`, `missing_indicator`, and `indicator_code`), and no `logit_code` or `prevalence_code` variables.

In [ ]:
transform_thin = vtreat.BinomialOutcomeTreatment(
    outcome_name='yc',    # outcome variable
    outcome_target=True,  # outcome of interest
    cols_to_copy=['y'],   # columns to "carry along" but not treat as input variables
    params = vtreat.vtreat_parameters({
        'filter_to_recommended': False,
        'coders': {'clean_copy',
                   'missing_indicator',
                   'indicator_code',
                  }
    })
)

transform_thin.fit_transform(d, d['yc']).head()

In [ ]:
transform_thin.score_frame_

## Deriving the Default Thresholds

While machine learning algorithms are generally tolerant to a reasonable number of irrelevant or noise variables, too many irrelevant variables can lead to serious overfit; see [this article](http://www.win-vector.com/blog/2014/02/bad-bayes-an-example-of-why-you-need-hold-out-testing/) for an extreme example, one we call "Bad Bayes". The default threshold is an attempt to eliminate obviously irrelevant variables early.

Imagine that you have a pure noise dataset, where none of the *n* inputs are related to the output. If you treat each variable as a one-variable model for the output, and look at the significances of each model, these significance-values will be uniformly distributed in the range [0:1]. You want to pick a weakest possible significance threshold that eliminates as many noise variables as possible. A moment's thought should convince you that a threshold of *1/n* allows only one variable through, in expectation. 

This leads to the general-case heuristic that a significance threshold of *1/n* on your variables should allow only one irrelevant variable through, in expectation (along with all the relevant variables). Hence, *1/n* used to be our recommended threshold, when we developed the R version of `vtreat`.

We noticed, however, that this biases the filtering against numerical variables, since there are at most two derived variables (of types *clean_copy* and *missing_indicator* for every numerical variable in the original data. Categorical variables, on the other hand, are expanded to many derived variables: several indicators (one for every common level), plus a *logit_code* and a *prevalence_code*. So we now reweight the thresholds. 

Suppose you have a (treated) data set with *ntreat* different types of `vtreat` variables (`clean_copy`, `indicator_code`, etc).
There are *nT* variables of type *T*. Then the default threshold for all the variables of type *T* is *1/(ntreat nT)*. This reweighting  helps to reduce the bias against any particular type of variable. The heuristic is still that the set of recommended variables will allow at most one noise variable into the set of candidate variables.

As noted above, because `vtreat` estimates variable significances using linear methods by default, some variables with a non-linear relationship  to the output may fail to pass the threshold. Setting the `filter_to_recommended` parameter to False will keep all derived variables in the treated frame, for the data scientist to filter (or not) as they will.



## Conclusion

In all cases (classification, regression, unsupervised, and multinomial classification) the intent is that `vtreat` transforms are essentially one liners.

The preparation commands are organized as follows:

 * **Regression**: [`R` regression example](https://github.com/WinVector/vtreat/blob/master/Examples/Regression/Regression.md), [`Python` regression example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Regression/Regression.md).
 * **Classification**: [`R` classification example](https://github.com/WinVector/vtreat/blob/master/Examples/Classification/Classification.md), [`Python` classification  example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/Multinomial.md).
 * **Unsupervised tasks**: [`R` unsupervised example](https://github.com/WinVector/vtreat/blob/master/Examples/Unsupervised/Unsupervised.md), [`Python` unsupervised example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Unsupervised/Unsupervised.md).
 * **Multinomial classification**: [`R` multinomial classification example](https://winvector.github.io/vtreat/articles/MultiClassVtreat.html), [`Python` multinomial classification example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/MultinomialExample.ipynb).

These current revisions of the examples are designed to be small, yet complete.  So as a set they have some overlap, but the user can rely mostly on a single example for a single task type.
